# Day 8: Seven Segment Search
https://adventofcode.com/2021/day/8

### Part 1

For now, focus on the easy digits. Consider this larger example:

Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [1]:
from typing import Optional
from advent_of_code import utils

In [2]:
def parse_line(line):
    """
    Parse a single input line,
    Return
        List of Tuple of sorted digits for input and
        List of Tuple of sorted digits for readout
    """
    input_str, readout_str = line.strip().split(" | ")

    input = [tuple(sorted(digit)) for digit in input_str.split(" ")]
    readout = [tuple(sorted(digit)) for digit in readout_str.split(" ")]

    return input, readout


def determine_uniques(sig_sets: list[tuple]) -> dict:
    """Return deterministic segment to number mappings."""
    d: dict = {}
    for s in sig_sets:
        if len(s) == 2:
            d[s] = 1
        elif len(s) == 3:
            d[s] = 7
        elif len(s) == 4:
            d[s] = 4
        elif len(s) == 7:
            d[s] = 8
    return d


def intersection_size(s1: tuple, s2: tuple) -> int:
    """returns length of intersection set between two strings."""
    return len(set(s1).intersection(s2))


def value_to_key(mapping: dict, value: int) -> Optional[tuple]:
    """Given a value in a dictionary we return the first match"""
    return next(
        (signal for signal, num in mapping.items() if num == value), None
    )


def determine_length_six(mapping, sig_sets):
    """Of length 6, se have 0, 6, and 9. Find matches and set mapping."""

    for s in sig_sets:
        if len(s) != 6 or s in mapping.keys():
            continue

        if (
            value_to_key(mapping, 1)
            and intersection_size(value_to_key(mapping, 1), s) == 1
        ):
            mapping[s] = 6

        elif (
            value_to_key(mapping, 4)
            and intersection_size(value_to_key(mapping, 4), s) == 4
        ):
            mapping[s] = 9

        elif (
            value_to_key(mapping, 4)
            and intersection_size(value_to_key(mapping, 4), s) == 3
            and value_to_key(mapping, 1)
            and intersection_size(value_to_key(mapping, 1), s) == 2
        ):
            mapping[s] = 0


def determine_length_five(mapping, sig_sets):
    """Of length 5, se have 2, 3, and 5. Find matches and set mapping."""

    for s in sig_sets:
        if len(s) != 5 or s in mapping.keys():
            continue

        if (
            value_to_key(mapping, 4)
            and intersection_size(value_to_key(mapping, 4), s) == 2
        ):
            mapping[s] = 2

        elif (
            value_to_key(mapping, 7)
            and intersection_size(value_to_key(mapping, 7), s) == 3
        ):
            mapping[s] = 3

        elif (
            value_to_key(mapping, 4)
            and intersection_size(value_to_key(mapping, 4), s) == 3
        ):
            mapping[s] = 5


def readout_sig_to_num(mapping, readouts) -> int:
    """convert readout signals into their numbers"""
    readout_num = 0
    for s in readouts:
        readout_num = readout_num * 10 + mapping.get(s)
    return readout_num



In [3]:
# Run test input and ensure it gives the correct answer.
sum = 0
with open(input_file) as f:
    for line in f:
        _, readouts = parse_line(line)
        mapping: dict = determine_uniques(readouts)
        sum += len([r for r in readouts if r in mapping.keys()])
assert sum == 26

# Run "production" input and print answer.
input_file = utils.input_location(day=8)
sum = 0
with open(input_file) as f:
    for line in f:
        _, readouts = parse_line(line)
        mapping: dict = determine_uniques(readouts)
        sum += len([r for r in readouts if r in mapping.keys()])
sum

470

### Part 2

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [4]:
def process_input(input_file: str) -> int:
    """returns sum of readouts, i.e., answer to part two"""

    sum_all = 0
    with open(input_file) as f:
        for line in f:
            inputs, readouts = parse_line(line)

            # d in form -> tuple(sorted signal): readout int
            # determine 1, 4, 7, 8 based on their length being unique.
            mapping: dict = determine_uniques(inputs + readouts)

            # determine the 6 length numbers 0, 6, 9
            determine_length_six(mapping, inputs + readouts)

            # determine the 5 length numbers 2, 3, 5
            determine_length_five(mapping, inputs + readouts)

            values = [
                value_to_key(mapping, 0),
                value_to_key(mapping, 1),
                value_to_key(mapping, 2),
                value_to_key(mapping, 3),
                value_to_key(mapping, 4),
                value_to_key(mapping, 5),
                value_to_key(mapping, 6),
                value_to_key(mapping, 7),
                value_to_key(mapping, 8),
                value_to_key(mapping, 9),
            ]

            if any(values) is None:
                print(values)
                raise ValueError("One of the values was not mapped.")

            readout = readout_sig_to_num(mapping, readouts)
            sum_all += readout

    return sum_all

In [5]:
# Run test input and assert the answer.
input_file = utils.test_input_location(day=8)
assert process_input(input_file) == 61229

# Run "production" input and print the answer.
input_file = utils.input_location(day=8)
process_input(input_file)

989396